In [1]:
# Imports and Base Image

import kfp
import kfp.dsl as dsl
from kfp.dsl import Input, Output
from kfp.dsl import Dataset, Artifact
from kfp.dsl import Model, Metrics, ClassificationMetrics

from typing import NamedTuple

BASE_IMAGE = 'nvcr.io/nvidia/tensorflow:25.01-tf2-py3'


In [2]:
# load data

@dsl.component(
    base_image=BASE_IMAGE,
)
def load_data(
    x_train_pickle: Output[Dataset],
    y_train_pickle: Output[Dataset],
    x_test_pickle: Output[Dataset],
    y_test_pickle: Output[Dataset],
):
    # import dataset
    from keras.datasets import mnist
    import numpy as np
    import pickle

    # load dataset
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    # count the number of unique train labels
    unique, counts = np.unique(y_train, return_counts=True)
    print("Train labels: ", dict(zip(unique, counts)))

    # count the number of unique test labels
    unique, counts = np.unique(y_test, return_counts=True)
    print("\nTest labels: ", dict(zip(unique, counts)))

    with open(x_train_pickle.path, "wb") as file:
        pickle.dump(x_train, file)

    with open(y_train_pickle.path, "wb") as file:
        pickle.dump(y_train, file)

    with open(x_test_pickle.path, "wb") as file:
        pickle.dump(x_test, file)

    with open(y_test_pickle.path, "wb") as file:
        pickle.dump(y_test, file)

In [3]:
# Preprocess data 

@dsl.component(
    base_image=BASE_IMAGE
)
def preprocess_data(
    x_train_pickle: Input[Dataset],
    y_train_pickle: Input[Dataset],
    x_test_pickle: Input[Dataset],
    y_test_pickle: Input[Dataset],
    x_train_prep: Output[Dataset], # Output preprocessed training data (needed for quantization)
    y_train_prep: Output[Dataset],
    x_test_prep: Output[Dataset],
    y_test_prep: Output[Dataset],
) -> NamedTuple("outputs", input_size=int, num_labels=int):

    from keras.utils import to_categorical
    import numpy as np
    import pickle
    from typing import NamedTuple

    with open(x_train_pickle.path, "rb") as file:
        x_train = pickle.load(file)

    with open(y_train_pickle.path, "rb") as file:
        y_train = pickle.load(file)

    with open(x_test_pickle.path, "rb") as file:
        x_test = pickle.load(file)

    with open(y_test_pickle.path, "rb") as file:
        y_test = pickle.load(file)

    num_labels = len(np.unique(y_train))

    # Reshape and normalize training data (needed for quantization calibration)
    image_size = x_train.shape[1]
    input_size = image_size * image_size
    x_train_processed = np.reshape(x_train, [-1, input_size])
    x_train_processed = x_train_processed.astype("float32") / 255

    # Reshape and normalize test data
    x_test_processed = np.reshape(x_test, [-1, input_size])
    x_test_processed = x_test_processed.astype("float32") / 255

    # One-hot encode labels
    y_train_processed = to_categorical(y_train)
    y_test_processed = to_categorical(y_test)

    # Save processed data
    with open(x_train_prep.path, "wb") as file:
        pickle.dump(x_train_processed, file)

    with open(y_train_prep.path, "wb") as file:
        pickle.dump(y_train_processed, file)

    with open(x_test_prep.path, "wb") as file:
        pickle.dump(x_test_processed, file)

    with open(y_test_prep.path, "wb") as file:
        pickle.dump(y_test_processed, file)

    outputs = NamedTuple("outputs", input_size=int, num_labels=int)
    return outputs(input_size, num_labels)


In [4]:
# Train the model

@dsl.component(
    base_image=BASE_IMAGE
)
def train(
    input_size: int,
    num_labels: int,
    epochs: int,
    x_train_prep: Input[Dataset], # Changed input name for clarity
    y_train_prep: Input[Dataset], # Changed input name for clarity
    model_artifact: Output[Model],
    log: Output[Artifact],
):
    from keras.callbacks import TensorBoard
    from keras.models import Sequential
    from keras.layers import Dense, Activation, Dropout
    import pickle
    import os
    from datetime import datetime

    import tensorflow as tf
    gpus = tf.config.list_physical_devices('GPU')
    if not gpus:
        print("TensorFlow CANNOT see any GPUs. GPU acceleration is NOT possible.")
    else:
        print(f"TensorFlow found {len(gpus)} GPU(s): {gpus}")

    batch_size = 128
    hidden_units = 256
    dropout = 0.45

    with open(x_train_prep.path, "rb") as file: # Changed path name
        x_train = pickle.load(file)

    with open(y_train_prep.path, "rb") as file: # Changed path name
        y_train = pickle.load(file)


    log_dir = f"{log.path}/logs/fit/{datetime.now().strftime('%Y%m%d-%H%M%S')}"
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    model = Sequential()
    model.add(Dense(hidden_units, input_dim=input_size))
    model.add(Activation("relu"))
    model.add(Dropout(dropout))
    model.add(Dense(hidden_units))
    model.add(Activation("relu"))
    model.add(Dropout(dropout))
    model.add(Dense(num_labels))
    model.add(Activation("softmax"))

    model.summary()

    model.compile(
        loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )

    model.fit(
        x_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[tensorboard_callback],
    )

    # Save the Keras model in the native format
    os.makedirs(model_artifact.path, exist_ok=True)
    # Keras model needs to be saved in a sub-directory for TFLite conversion later
    model_dir_path = os.path.join(model_artifact.path, "fp32_model")
    os.makedirs(model_dir_path, exist_ok=True)
    model_path = os.path.join(model_dir_path, "model.keras") # Use .keras extension
    model.save(model_path)
    print(f"[train] Keras model saved to {model_path}")

In [5]:
# Quantize the model

@dsl.component(
    base_image=BASE_IMAGE,
)
def quantize_model(
    keras_model_input: Input[Model], # Input is the directory containing the Keras model
    x_train_prep: Input[Dataset], # Need training data for calibration
    quantized_model_output: Output[Model], # Output will be the .tflite file
):
    import tensorflow as tf
    import numpy as np
    import pickle
    import os

    # Load the FP32 Keras model
    # model.save() creates a directory, find the .keras file inside
    model_dir_path = os.path.join(keras_model_input.path, "fp32_model")
    keras_model_path = os.path.join(model_dir_path, "model.keras")
    print(f"[quantize] Loading Keras model from: {keras_model_path}")
    model = tf.keras.models.load_model(keras_model_path)

    # Load representative dataset (calibration data)
    print(f"[quantize] Loading representative data from: {x_train_prep.path}")
    with open(x_train_prep.path, "rb") as file:
        x_train = pickle.load(file)

    # Create a representative dataset generator
    # Use a subset for faster calibration (e.g., 100 samples)
    def representative_data_gen():
        num_samples = min(100, len(x_train))
        for i in range(num_samples):
            # Get sample input data as a numpy array
            # Input shape needs to match model input: add batch dimension
            yield [np.expand_dims(x_train[i], axis=0).astype(np.float32)]

    print("[quantize] Starting TFLite conversion with INT8 quantization...")
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_data_gen
    # Ensure that if any ops can't be quantized, the converter throws an error
    # Ensure ops are compatible with integer-only inference
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    # Set input and output tensors to int8 (or uint8)
    # MNIST inputs are 0-255 originally, normalized to 0-1.
    # int8 quantization typically maps float range [-a, a] to int [-128, 127].
    # uint8 quantization typically maps float range [0, b] to uint [0, 255].
    # Let's try int8 as it's common, TF Lite handles the scaling.
    converter.inference_input_type = tf.int8
    converter.inference_output_type = tf.int8

    tflite_model_quant = converter.convert()
    print("[quantize] TFLite conversion finished.")

    # Save the quantized model
    # The output artifact path is a directory, save the file inside it
    os.makedirs(quantized_model_output.path, exist_ok=True)
    tflite_model_path = os.path.join(quantized_model_output.path, 'quantized_model.tflite')
    with open(tflite_model_path, 'wb') as f:
        f.write(tflite_model_quant)
    print(f"[quantize] Quantized TFLite model saved to: {tflite_model_path}")

In [6]:
# Evaluate fp32 model

@dsl.component(
    base_image=BASE_IMAGE,
    packages_to_install=["scikit-learn"],
)
def evaluate_fp32_model(
    model_artifact: Input[Model], # Input is the directory containing the Keras model
    metrics: Output[ClassificationMetrics],
    scalar_metrics: Output[Metrics],
    x_test_prep: Input[Dataset], # Changed input name for clarity
    y_test_prep: Input[Dataset], # Changed input name for clarity
):
    from keras.models import load_model
    from keras.metrics import Precision # Note: Keras Precision might not be ideal here
    from sklearn.metrics import confusion_matrix, accuracy_score, precision_score
    import numpy as np
    import os
    import pickle

    # Load the Keras model
    model_dir_path = os.path.join(model_artifact.path, "fp32_model")
    model_path = os.path.join(model_dir_path, "model.keras")
    print(f"[evaluate_fp32] loading model from {model_path}")
    model = load_model(model_path)

    batch_size = 128

    with open(x_test_prep.path, "rb") as file: # Changed path name
        x_test = pickle.load(file)

    with open(y_test_prep.path, "rb") as file: # Changed path name
        y_test = pickle.load(file) # y_test is one-hot encoded

    # Evaluate using Keras model.evaluate
    loss, acc = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=0)
    print(f"[evaluate_fp32] Keras Evaluation - Loss: {loss:.4f}, Accuracy: {acc:.4f}")

    # Get predictions for confusion matrix and other metrics
    y_pred_proba = model.predict(x_test, batch_size=batch_size)
    y_pred_labels = np.argmax(y_pred_proba, axis=1)
    y_true_labels = np.argmax(y_test, axis=1)

    # Calculate metrics using sklearn
    # Use macro average for precision as it's multi-class
    precision = precision_score(y_true_labels, y_pred_labels, average='macro', zero_division=0)
    print(f"[evaluate_fp32] Sklearn Metrics - Accuracy: {acc:.4f}, Precision (macro): {precision:.4f}")

    # Log metrics
    scalar_metrics.log_metric("fp32_accuracy", float(acc))
    scalar_metrics.log_metric("fp32_loss", float(loss))
    scalar_metrics.log_metric("fp32_precision_macro", float(precision))

    metrics.log_confusion_matrix(
        [str(i) for i in range(10)], # Class names '0' through '9'
        confusion_matrix(y_true_labels, y_pred_labels).tolist(), # Convert np array to list
    )
    print("[evaluate_fp32] Metrics logged.")

In [7]:
# Evaluate quantized model

@dsl.component(
    base_image=BASE_IMAGE,
    packages_to_install=["scikit-learn"],
)
def evaluate_quantized_model(
    quantized_model_artifact: Input[Model], # Input is the directory containing .tflite
    metrics: Output[ClassificationMetrics],
    scalar_metrics: Output[Metrics],
    x_test_prep: Input[Dataset],
    y_test_prep: Input[Dataset],
):
    import tensorflow as tf
    import numpy as np
    import pickle
    import os
    from sklearn.metrics import confusion_matrix, accuracy_score, precision_score

    # Load the TFLite model and allocate tensors
    tflite_model_path = os.path.join(quantized_model_artifact.path, 'quantized_model.tflite')
    print(f"[evaluate_quantized] Loading TFLite model from: {tflite_model_path}")
    interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()

    # Get input and output tensor details
    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]
    input_dtype = input_details['dtype']
    output_dtype = output_details['dtype']
    input_scale, input_zero_point = input_details["quantization"]
    output_scale, output_zero_point = output_details["quantization"]
    print(f"[evaluate_quantized] Input Details: {input_details}")
    print(f"[evaluate_quantized] Output Details: {output_details}")

    # Load test data
    print(f"[evaluate_quantized] Loading test data...")
    with open(x_test_prep.path, "rb") as file:
        x_test_float = pickle.load(file) # Load original float32 data

    with open(y_test_prep.path, "rb") as file:
        y_test_one_hot = pickle.load(file)
    y_true_labels = np.argmax(y_test_one_hot, axis=1)

    print(f"[evaluate_quantized] Quantizing input data to {input_dtype}...")
    # Quantize input data according to input tensor details
    # Formula: int_value = float_value / scale + zero_point
    x_test_quantized = (x_test_float / input_scale) + input_zero_point
    x_test_quantized = x_test_quantized.astype(input_dtype)
    print(f"[evaluate_quantized] Input data shape: {x_test_quantized.shape}, dtype: {x_test_quantized.dtype}")


    # Run inference
    print(f"[evaluate_quantized] Running inference on {len(x_test_quantized)} samples...")
    y_pred_quantized_list = []
    for i in range(len(x_test_quantized)):
        interpreter.set_tensor(input_details['index'], np.expand_dims(x_test_quantized[i], axis=0))
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details['index'])
        y_pred_quantized_list.append(output_data[0]) # Remove batch dim

    y_pred_quantized = np.array(y_pred_quantized_list)
    print(f"[evaluate_quantized] Inference complete. Output shape: {y_pred_quantized.shape}, dtype: {y_pred_quantized.dtype}")

    # Dequantize output predictions to calculate loss/metrics easily
    # Formula: float_value = (int_value - zero_point) * scale
    y_pred_float = (y_pred_quantized.astype(np.float32) - output_zero_point) * output_scale
    print(f"[evaluate_quantized] Dequantized output shape: {y_pred_float.shape}, dtype: {y_pred_float.dtype}")

    # Get predicted labels
    y_pred_labels = np.argmax(y_pred_float, axis=1)

    # Calculate metrics
    accuracy = accuracy_score(y_true_labels, y_pred_labels)
    # Use macro average for precision as it's multi-class
    precision = precision_score(y_true_labels, y_pred_labels, average='macro', zero_division=0)
    # Calculate categorical cross-entropy loss (requires probabilities)
    # Softmax the dequantized outputs
    def softmax(x):
        e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return e_x / e_x.sum(axis=1, keepdims=True)

    y_pred_proba = softmax(y_pred_float)
    # Compute loss - use stable version
    N = y_pred_proba.shape[0]
    log_likelihood = -np.log(y_pred_proba[range(N), y_true_labels] + 1e-9) # Add epsilon for stability
    loss = np.sum(log_likelihood) / N

    print(f"[evaluate_quantized] Metrics - Accuracy: {accuracy:.4f}, Precision (macro): {precision:.4f}, Loss: {loss:.4f}")

    # Log metrics
    scalar_metrics.log_metric("quantized_int8_accuracy", float(accuracy))
    scalar_metrics.log_metric("quantized_int8_loss", float(loss))
    scalar_metrics.log_metric("quantized_int8_precision_macro", float(precision))

    metrics.log_confusion_matrix(
        [str(i) for i in range(10)], # Class names '0' through '9'
        confusion_matrix(y_true_labels, y_pred_labels).tolist(), # Convert np array to list
    )
    print("[evaluate_quantized] Metrics logged.")

In [8]:
@dsl.component(
    base_image=BASE_IMAGE,
    packages_to_install=["scikit-learn"],
)
def evaluate_quantized_model(
    quantized_model_artifact: Input[Model], # Input is the directory containing .tflite
    metrics: Output[ClassificationMetrics],
    scalar_metrics: Output[Metrics],
    x_test_prep: Input[Dataset],
    y_test_prep: Input[Dataset],
):
    import tensorflow as tf
    import numpy as np
    import pickle
    import os
    from sklearn.metrics import confusion_matrix, accuracy_score, precision_score

    # Load the TFLite model and allocate tensors
    tflite_model_path = os.path.join(quantized_model_artifact.path, 'quantized_model.tflite')
    print(f"[evaluate_quantized] Loading TFLite model from: {tflite_model_path}")
    interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()

    # Get input and output tensor details
    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]
    input_dtype = input_details['dtype']
    output_dtype = output_details['dtype']
    input_scale, input_zero_point = input_details["quantization"]
    output_scale, output_zero_point = output_details["quantization"]
    print(f"[evaluate_quantized] Input Details: {input_details}")
    print(f"[evaluate_quantized] Output Details: {output_details}")

    # Load test data
    print(f"[evaluate_quantized] Loading test data...")
    with open(x_test_prep.path, "rb") as file:
        x_test_float = pickle.load(file) # Load original float32 data

    with open(y_test_prep.path, "rb") as file:
        y_test_one_hot = pickle.load(file)
    y_true_labels = np.argmax(y_test_one_hot, axis=1)

    print(f"[evaluate_quantized] Quantizing input data to {input_dtype}...")
    # Quantize input data according to input tensor details
    # Formula: int_value = float_value / scale + zero_point
    x_test_quantized = (x_test_float / input_scale) + input_zero_point
    x_test_quantized = x_test_quantized.astype(input_dtype)
    print(f"[evaluate_quantized] Input data shape: {x_test_quantized.shape}, dtype: {x_test_quantized.dtype}")


    # Run inference
    print(f"[evaluate_quantized] Running inference on {len(x_test_quantized)} samples...")
    y_pred_quantized_list = []
    for i in range(len(x_test_quantized)):
        interpreter.set_tensor(input_details['index'], np.expand_dims(x_test_quantized[i], axis=0))
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details['index'])
        y_pred_quantized_list.append(output_data[0]) # Remove batch dim

    y_pred_quantized = np.array(y_pred_quantized_list)
    print(f"[evaluate_quantized] Inference complete. Output shape: {y_pred_quantized.shape}, dtype: {y_pred_quantized.dtype}")

    # Dequantize output predictions to calculate loss/metrics easily
    # Formula: float_value = (int_value - zero_point) * scale
    y_pred_float = (y_pred_quantized.astype(np.float32) - output_zero_point) * output_scale
    print(f"[evaluate_quantized] Dequantized output shape: {y_pred_float.shape}, dtype: {y_pred_float.dtype}")

    # Get predicted labels
    y_pred_labels = np.argmax(y_pred_float, axis=1)

    # Calculate metrics
    accuracy = accuracy_score(y_true_labels, y_pred_labels)
    # Use macro average for precision as it's multi-class
    precision = precision_score(y_true_labels, y_pred_labels, average='macro', zero_division=0)
    # Calculate categorical cross-entropy loss (requires probabilities)
    # Softmax the dequantized outputs
    def softmax(x):
        e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return e_x / e_x.sum(axis=1, keepdims=True)

    y_pred_proba = softmax(y_pred_float)
    # Compute loss - use stable version
    N = y_pred_proba.shape[0]
    log_likelihood = -np.log(y_pred_proba[range(N), y_true_labels] + 1e-9) # Add epsilon for stability
    loss = np.sum(log_likelihood) / N

    print(f"[evaluate_quantized] Metrics - Accuracy: {accuracy:.4f}, Precision (macro): {precision:.4f}, Loss: {loss:.4f}")

    # Log metrics
    scalar_metrics.log_metric("quantized_int8_accuracy", float(accuracy))
    scalar_metrics.log_metric("quantized_int8_loss", float(loss))
    scalar_metrics.log_metric("quantized_int8_precision_macro", float(precision))

    metrics.log_confusion_matrix(
        [str(i) for i in range(10)], # Class names '0' through '9'
        confusion_matrix(y_true_labels, y_pred_labels).tolist(), # Convert np array to list
    )
    print("[evaluate_quantized] Metrics logged.")

In [9]:
# Create pipeline

@dsl.pipeline(
    name="mnist-pipeline-tf-gpu-quantized", # Renamed pipeline
    description="Loads MNIST, preprocesses, trains, quantizes, and evaluates FP32 and INT8 models."
)
def mnist_pipeline(epochs: int = 20): # Default epochs set
    # Load Data
    load_data_task = (
        load_data()
        .set_memory_limit("4G").set_memory_request("2G")
        .set_cpu_limit("2").set_cpu_request("1")
        .set_display_name("Load MNIST Data")
        .set_caching_options(False) # Disable caching temporarily for troubleshooting
    )

    # Preprocess Data
    preprocess_task = (
        preprocess_data(
            x_train_pickle=load_data_task.outputs["x_train_pickle"],
            y_train_pickle=load_data_task.outputs["y_train_pickle"],
            x_test_pickle=load_data_task.outputs["x_test_pickle"],
            y_test_pickle=load_data_task.outputs["y_test_pickle"],
        )
        .set_memory_limit("4G").set_memory_request("2G")
        .set_cpu_limit("1").set_cpu_request("1")
        .set_display_name("Preprocess Data")
        .set_caching_options(False) # Disable caching temporarily for troubleshooting
    )
    preprocess_task.after(load_data_task) # Ensure execution order

    # Train FP32 Model
    train_task = (
        train(
            input_size=preprocess_task.outputs["input_size"],
            num_labels=preprocess_task.outputs["num_labels"],
            epochs=epochs,
            x_train_prep=preprocess_task.outputs["x_train_prep"], # Use preprocessed data
            y_train_prep=preprocess_task.outputs["y_train_prep"], # Use preprocessed data
        )
        .set_gpu_limit(1)
        # Use the correct node selector syntax if needed, or remove if not required
        # .add_node_selector_constraint('nvidia.com/gpu', 'true') # Example if value is 'true'
        # .add_node_selector_constraint('nvidia.com/gpu', '<gpu_model_name>') # Example if value is model name
        # Or remove if set_gpu_limit is sufficient
        .set_memory_limit("4G").set_memory_request("2G") # Increased memory for TF
        .set_cpu_limit("2").set_cpu_request("1") # Increased CPU slightly
        .set_display_name("Train FP32 Model")
        .set_caching_options(False) # Disable caching temporarily for troubleshooting
    )
    train_task.after(preprocess_task)

    # Evaluate FP32 Model
    evaluate_fp32_task = (
        evaluate_fp32_model( # Renamed function call
            model_artifact=train_task.outputs["model_artifact"],
            x_test_prep=preprocess_task.outputs["x_test_prep"], # Use preprocessed data
            y_test_prep=preprocess_task.outputs["y_test_prep"], # Use preprocessed data
        )
        .set_memory_limit("4G").set_memory_request("2G")
        .set_cpu_limit("1").set_cpu_request("1")
        .set_display_name("Evaluate FP32 Model")
        .set_caching_options(False) # Disable caching temporarily for troubleshooting
    )
    evaluate_fp32_task.after(train_task)

    # Quantize Model to INT8
    quantize_task = (
        quantize_model(
            keras_model_input=train_task.outputs["model_artifact"],
            x_train_prep=preprocess_task.outputs["x_train_prep"], # Use preprocessed train data for calibration
        )
        .set_memory_limit("4G").set_memory_request("2G") # Quantization can use memory
        .set_cpu_limit("1").set_cpu_request("1")
        .set_display_name("Quantize Model (INT8)")
        .set_caching_options(False) # Disable caching temporarily for troubleshooting
    )
    quantize_task.after(train_task) # Depends on the trained model

    # Evaluate Quantized INT8 Model
    evaluate_quantized_task = (
        evaluate_quantized_model(
            quantized_model_artifact=quantize_task.outputs["quantized_model_output"],
            x_test_prep=preprocess_task.outputs["x_test_prep"], # Use preprocessed test data
            y_test_prep=preprocess_task.outputs["y_test_prep"], # Use preprocessed test data
        )
        .set_memory_limit("4G").set_memory_request("2G")
        .set_cpu_limit("1").set_cpu_request("1")
        .set_display_name("Evaluate Quantized INT8 Model")
        .set_caching_options(False) # Disable caching temporarily for troubleshooting
    )
    # Depends on quantized model and preprocessed data, run after quantization
    evaluate_quantized_task.after(quantize_task)
    evaluate_quantized_task.after(preprocess_task) # Explicit dependency for clarity


In [10]:
# Run pipeline (
client = kfp.Client()

# Make sure KFP client points to your Kubeflow endpoint if needed
# client = kfp.Client(host='<your-kubeflow-pipelines-url>')

run = client.create_run_from_pipeline_func(
    mnist_pipeline,
    arguments={"epochs": 3},
    experiment_name="mnist_pipeline",
)

print(f"Pipeline run submitted. Run details: {run.run_id}")


/opt/conda/lib/python3.11/site-packages/kfp/client/client.py:159: FutureWarning: This client only works with Kubeflow Pipeline v2.0.0-beta.2 and later versions.
  warnings.warn(


Pipeline run submitted. Run details: f2f7d428-ec37-42d3-84ee-7d9319989667
